De donde obtener la data

- yahoo finance
- 

# with Yahoo Finance

In [1]:
start = "2022-01-01"

In [40]:

import yfinance as yf
symbol = "AAPL"
data = yf.download(symbol, start=start)

[*********************100%%**********************]  1 of 1 completed


# With CCXT

In [1]:
import ccxt
import pandas as pd
from datetime import datetime 
import time


In [3]:
symbol = "BTCUSDT"
ex = ccxt.binance() # https://docs.ccxt.com/#/exchanges/binance?id=fetchohlcv
timeframe = "5m"

In [7]:

# simple fetch only 1000
fetchData = ex.fetch_ohlcv(
    symbol, 
    timeframe, 
    limit=None
    )

In [4]:
# get unlimited data

import ccxt
import pandas as pd
from datetime import datetime 

symbol = "BTCUSDT"
ex = ccxt.binance() # https://docs.ccxt.com/#/exchanges/binance?id=fetchohlcv
timeframe = "5m"

from_ts = ex.parse8601('2024-03-15 00:00:00')
max = 10000
ohlcv_list = []
ohlcv = ex.fetch_ohlcv(
    symbol, 
    timeframe, 
    since=from_ts, 
    limit=1000)
ohlcv_list.append(ohlcv)

while True:
   
    from_ts = ohlcv[-1][0]
    new_ohlcv = ex.fetch_ohlcv(
        symbol, 
        timeframe,
        since=from_ts, 
        limit=1000
        )
    
    ohlcv.extend(new_ohlcv)

    if len(new_ohlcv)!=1000:
        break
    
fetchData = ohlcv

len(fetchData)

ExchangeNotAvailable: binance GET https://api.binance.com/api/v3/exchangeInfo 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}

In [6]:
import requests
import pandas as pd
import time

def get_binance_futures_data(symbol, interval, start_time, end_time, limit=1500):
    """
    Descarga datos históricos de futuros de Binance para un símbolo e intervalo especificados.
    
    Parámetros:
    symbol (str): El símbolo de trading, por ejemplo, 'BTCUSDT'.
    interval (str): El intervalo de tiempo de las velas, por ejemplo, '5m'.
    start_time (int): El tiempo de inicio en milisegundos desde la época Unix.
    end_time (int): El tiempo de finalización en milisegundos desde la época Unix.
    limit (int): El número máximo de datos por solicitud (máximo 1500).

    Retorna:
    pd.DataFrame: Un DataFrame con los datos históricos.
    """
    url = "https://fapi.binance.com/fapi/v1/klines"
    data = []
    
    while start_time < end_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': limit
        }
        response = requests.get(url, params=params)
        print(response)
        response.raise_for_status()
        klines = response.json()

        if not klines:
            break

        data.extend(klines)
        start_time = klines[-1][0] + 1

        # Para evitar el rate limit de Binance
        time.sleep(0.1)

    columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 
               'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 
               'Taker Buy Quote Asset Volume', 'Ignore']
    
    df = pd.DataFrame(data, columns=columns)
    
    # Convertir los tiempos de milisegundos a datetime
    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
    
    return df

# Parámetros
symbol = 'BTCUSDT'
interval = '5m'
start_time = int(pd.Timestamp('2023-01-01').timestamp() * 1000)
end_time = int(pd.Timestamp('2023-06-01').timestamp() * 1000)

# Obtener datos
df = get_binance_futures_data(symbol, interval, start_time, end_time)

# Mostrar los primeros datos
print(df.head())


<Response [451]>


HTTPError: 451 Client Error:  for url: https://fapi.binance.com/fapi/v1/klines?symbol=BTCUSDT&interval=5m&startTime=1672531200000&endTime=1685577600000&limit=1500

Save file

In [15]:
header = ["DateStamp", "open", "high", "low", "close", "volume"]
data = pd.DataFrame(fetchData, columns=header)

data["Date"] = pd.to_datetime(data["DateStamp"], unit="ms")
data = data.set_index("Date")
data = data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})

with open(symbol + ""+ timeframe+'.csv', 'w') as f:
    f.write(data.to_csv())
    
print(len(data))

26968
